# Reuters database: multiclass classification 

## 보도자료 분류 : 46개 서로 다른 토픽으로 분류되는 로이터 보도자료 데이터베이스

- 만약 각각의 데이터들이 하나의 카테고리에 해당하면 single-label, multiclass classification
- 만약 각각의 데이터들이 복수의 카테고리에 해당 (이 경우, topic)하면, multilabel, multiclass classification

### Reuters 데이터세트 로드 
- Reuters 데이터세트는 Keras 에 패키지 되어 같이 제공됨
- 46개 토픽당 최소 10개 이상의 example 이 제공됨
- num_words=10000 은 10,000개의 제일 많이 발생하는 단어로 제한을 둠

In [ ]:
from tensorflow.keras.datasets import reuters
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(
    num_words=10000)

- 8,982 개 train 데이터와 2,246 test 데이터

In [ ]:
len(train_data)

In [ ]:
len(test_data)

- train 데이터는 word indeces 의 정수 리스트들고 이루어져 있음

In [ ]:
len(test_data)

#### 인코딩 되어있는 보도자료 데이터를 디코딩
- word_index 는 단어를 정수 인덱스로 dictionary 형태로 mapping
- reverse_word_index 를 통해 index 를 단어로 mapping
- decode_review 에서 3개의 index 들은 시스템 내부에서 예약되어 있음: 0: "padding", 1: "start of sequence", 2: "unknown"

In [ ]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_newswire = " ".join([reverse_word_index.get(i - 3, "?") for i in
    train_data[0]])

- 아래 예에 해당하는 레이블은 토픽 인덱스 (0~45 까지의) 

In [ ]:
train_labels[10]

### 데이터 준비
#### input data 를 인코딩
- 리스틀 tensor 로 신경망에 주입
- 정수 리스트를 0 과 1 로 이루어진 one-hot vector 로 변환하여 인코딩
- 맨처음 results 를 0 으로 채워진 행렬을 생성
- "results[i, sequence] = 1" 를 통하여 특정 index 의 results 를 1 로 셋팅
- train 과 test 데이터를 벡터화

In [ ]:
import numpy as np
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

#### 레이블을 인코딩
- 레이블의 범위 0 ~ 45 (topics) 이므로 dimension=46

In [ ]:
def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results
one_hot_train_labels = to_one_hot(train_labels)
one_hot_test_labels = to_one_hot(test_labels)

- Keras 에 내장된 방법을 통해서도 one_hot_vector 를 만들 수 있다

In [ ]:
"""
from tensorflow.keras.utils import to_categorical
one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)
"""

- 이제 데이터는 신경망에 투입될 수 있게 준비 됨

### 모델 구축
- 보도자료 문서를 분류하여 46개의 토픽 중 어디에 해당하는지를 분류하는 문제
- 출력층은 46 차원의 class 를 다루어야 함
- 64 차원의 Dense 층을 relu 활성화 함수로 갖춘 2개의 stack 를 거쳐 마지막 층에 46 차원의 Dense 층에 softmax 활성화 함수로
- 46개 class 가 확률분포를 각각 갖게되고 결국 총 합은 1 이 된다.

### 모델 정의

In [ ]:
model = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(46, activation="softmax")
])

### 모델 컴파일
- 이 문제와 같은 categorical classification 문제에는 categorical_crossentropy loss 가 가장 좋은 선택
- 모델의 확률분포 출력과 레이블의 실제 분포 사이의 거리를 측정
- rmsprop 옵티마이저는 거의 모든 문제에 있어 좋은 default 선택
- accuracy 를 선언하여 훈련 동안 정확도를 모니터링

In [ ]:
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

## 모델 성능 검증
- train 데이터 중에 10000 개를 validate 용으로 별도 분할

In [ ]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

## 모델 Training
- 훈련을 20회 반복하면서 실행 (epoch)
- mini-batch 크기는 512 샘플
- 검증용으로 분할 했던 10000 개 샘플(x_val, y_val)에 대하여 정확도와 손실을 모니터

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

### Training 과 Validation loss 를 matplotlib 를 사용하여 plot

- 'bo' 는 blue dot 을 의미
- 'b'는 solid blue line 을 의미

In [ ]:
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

### Training 과 Validation accuracy
- plt.clf 는 clear the picture

In [ ]:
plt.clf()
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
plt.plot(epochs, acc, "bo", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

- training accuracy 는 지속적으로 증가하는 데 반해, validation accuracy 는 9 epoch 이후에 증가세가 멈춤
- 전형적인 overfitting 문제. 문제를 방지하기 위해 모델을 새롭게 9 epoch 까지만 정의하고 훈련 시행

In [ ]:
model = keras.Sequential([
  layers.Dense(64, activation="relu"),
  layers.Dense(64, activation="relu"),
  layers.Dense(46, activation="softmax")
])
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.fit(partial_x_train,
          partial_y_train,
          epochs=9,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(x_test, one_hot_test_labels)

- 마지막 results 는 아래와 같음

In [ ]:
results

-  80% 에 근접한 정확도를 보임
-  임의로 분류를 시행하면 몇 % 정확도를 보일까?

In [ ]:
import copy
test_labels_copy = copy.copy(test_labels)
np.random.shuffle(test_labels_copy)
hits_array = np.array(test_labels) == np.array(test_labels_copy)
float(np.sum(hits_array)) / len(test_labels)

- random classifier 는 19% 의 분류 정확도를 보이는 것을 감안할 때, 이번 결과는 상당히 좋은 것으로 보임

- 테스트 데이터를 통하여 topic 을 예측


In [ ]:
predictions = model.predict(x_test)

- prediction 의 각각의 entry 들은 46 길이의 벡터 

In [ ]:
predictions[0].shape

- 46 클래스의 확률분포를 나타내므로 전체 합은 1

In [ ]:
np.sum(predictions[0])

- 가장 높은 확률을 가진 class

In [ ]:
np.argmax(predictions[0])

- one-hot-encoding 말고 다른 방법으로 labels 와 loss 를 다루는 방법
- 레이블을 정수 텐서로 인코딩 하는 방법

In [ ]:
y_train = np.array(train_labels)
y_test = np.array(test_labels)

- 이 경우 유일하게 바뀌는 부분은 loss 함수의 선택
- 정수 레이블의 경우, sparse_categorical_crossentropy 를 사용해야 함

In [ ]:
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

### 충분히 큰 중간 층(layer) 의 중요성
- 최종 output 이 46 차원이어서 중간 층이 46 unit 보다 적은 경우를 피해야 한다고 설명하였는데
- 만약 46 보다 작은 예를 들어 4 unit 을 사용한다면 어떻게 될까?

In [ ]:
model = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(4, activation="relu"),
    layers.Dense(46, activation="softmax")
])
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.fit(partial_x_train,
          partial_y_train,
          epochs=20,
          batch_size=128,
          validation_data=(x_val, y_val))

- 모델은 71% validation accuracy 에서 정점을 찍고 8% 하강한다. 
- 이런 현상은 주로 너무 작은 차원의 중간 공간을 가지고 많은 정보를 압축하려고 시도 하는 것에 기인한다.
- 4차원의 작은 공간에 대부분의 많은 정보를 꾸겨넣으려고 하지만 모두 담을 수는 없는 것이다